In [1]:
# conda remove --name SCENIC --all
# conda create --name SCENIC python=3.6.3 
# conda activate SCENIC
# python -m pip install pandas
# git clone https://github.com/aertslab/pycisTopic.git
# cd pycisTopic
# python -m pip install .
# python -m pip install --upgrade pip setuptools
# python -m pip install scanpy
# python -m pip install  pyranges
# python -m pip install requests --ignore-installed certifi
# python -m pip install ray
# python -m pip install tmtoolkit
# python -m pip install gensim
# python -m pip install loomxpy
# python -m pip install pyrle
# python -m pip install polars
# python -m pip  install typing-extensions --upgrade
# python -m pip install pyarrow
# python -m pip install loompy
# python -m pip install pyscenic
# python -m pip install pickle5
# python -m pip install ipykernel
#  python -m pip install pyBigWig
# install jupyter-lab
# mv pycisTopic/ pycisTopic_repo
#git clone https://github.com/aertslab/scenicplus
#cd scenicplus
#pip install .



import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import scanpy
import os
import requests
import pyranges as pr
import pycisTopic
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.iterative_peak_calling import *
import anndata
import pickle
import pyarrow 
import re


def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
 
    for key, value in freq.items():
        print("% s : % d" % (key, value))
 

outDir="SCENIC_results_MARCH2024/"
tmpDir="/tmp/"
#ray.shutdown()

In [2]:
# read the annaData. This comes from ~/data/tasks/margo.montandon/Analysis/basic_analysis.Rmd conversion step
# inside R when saving Seurat object
# library(scater)
# SaveH5Seurat(sobj.merged,"~/data/tasks/margo.montandon/Analysis/sobj.merged")
# Convert("~/data/tasks/margo.montandon/Analysis/sobj.merged.h5seurat", dest = "h5ad")

cell_data=scanpy.read_h5ad("../sobj.merged.h5ad")


In [3]:
cell_data.gene_names=cell_data.var
cluster='cluster_'
cell_df=cell_data.obs
barcodes_names=list(cell_df.index)
cell_df['barcodes_names']=barcodes_names
barcodes=[re.sub(".df_", "", x) for x in barcodes_names]
cell_df['barcodes']=barcodes
cell_df['cellnames']=cell_df['barcodes'] + '-' + cell_df['orig.ident']
cell_df.index = cell_df['cellnames']
#cell_df.index

cell_data.index=cell_df.index 

#anndata.AnnData.to_df(cell_data.obs)

#variable = 'VSN_cell_type', -->seurat_clusters
#sample_id_col = 'VSN_sample_id', --> orig.ident
#cell_df.head

myclusterslist=['wsnn_res.0.1', 
                'wsnn_res.0.2',
                'wsnn_res.0.3', 
                'wsnn_res.0.4', 
                'wsnn_res.0.5', 
                'wsnn_res.0.6',
                'wsnn_res.0.7', 
                'wsnn_res.0.8', 
                'wsnn_res.0.9', 
                'wsnn_res.1',
                'wsnn_res.1.1', 
                'wsnn_res.1.2', 
                'wsnn_res.1.3', 
                'wsnn_res.1.4',
                'wsnn_res.1.5', 
                'wsnn_res.1.6', 
                'wsnn_res.1.7', 
                'wsnn_res.1.8',
                'wsnn_res.1.9', 
                'wsnn_res.2', 
                'seurat_clusters' ]

for items in myclusterslist:
    cluster='cluster_'
    cell_df[items]=cluster + cell_df[items].astype(str)
    
#resolution picked because heaviest file
resolution="wsnn_res.0.1"        
cell_df['clusters']=cluster + cell_df[resolution].astype(str)

In [4]:
type(cell_df)

pandas.core.frame.DataFrame

In [5]:
sample_id_col = 'orig.ident'
sample_ids = list(set(cell_df[sample_id_col]))
sample_ids 

['2dpf', '6dpf', '4dpf']

In [6]:

fragments_dict={'6dpf':"/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//6dpf/atac_fragments.tsv.gz",
                '4dpf':"/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//4dpf/atac_fragments.tsv.gz",
                '2dpf':"/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//2dpf/atac_fragments.tsv.gz"}
fragments_dict

{'6dpf': '/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//6dpf/atac_fragments.tsv.gz',
 '4dpf': '/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//4dpf/atac_fragments.tsv.gz',
 '2dpf': '/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon//countmatrix//2dpf/atac_fragments.tsv.gz'}

In [7]:
cell_df.columns

Index(['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_ATAC',
       'nFeature_ATAC', 'condition', 'nCount_SCT', 'nFeature_SCT',
       'SCT.weight', 'ATAC.weight', 'wsnn_res.0.1', 'wsnn_res.0.2',
       'wsnn_res.0.3', 'wsnn_res.0.4', 'wsnn_res.0.5', 'wsnn_res.0.6',
       'wsnn_res.0.7', 'wsnn_res.0.8', 'wsnn_res.0.9', 'wsnn_res.1',
       'wsnn_res.1.1', 'wsnn_res.1.2', 'wsnn_res.1.3', 'wsnn_res.1.4',
       'wsnn_res.1.5', 'wsnn_res.1.6', 'wsnn_res.1.7', 'wsnn_res.1.8',
       'wsnn_res.1.9', 'wsnn_res.2', 'seurat_clusters', 'barcodes_names',
       'barcodes', 'cellnames', 'clusters'],
      dtype='object')

In [8]:
cell_df

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_ATAC,nFeature_ATAC,condition,nCount_SCT,nFeature_SCT,SCT.weight,...,wsnn_res.1.6,wsnn_res.1.7,wsnn_res.1.8,wsnn_res.1.9,wsnn_res.2,seurat_clusters,barcodes_names,barcodes,cellnames,clusters
cellnames,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAACTAGGG-1-2dpf,2dpf,1781.0,796,2.919708,529.0,271,2dpf,1889.0,796,0.432149,...,cluster_1,cluster_1,cluster_1,cluster_1,cluster_1,cluster_1,2df_AAACAGCCAACTAGGG-1,AAACAGCCAACTAGGG-1,AAACAGCCAACTAGGG-1-2dpf,cluster_cluster_5
AAACAGCCATTATGGT-1-2dpf,2dpf,2521.0,976,3.292344,518.0,272,2dpf,2204.0,976,0.658799,...,cluster_0,cluster_0,cluster_0,cluster_0,cluster_0,cluster_0,2df_AAACAGCCATTATGGT-1,AAACAGCCATTATGGT-1,AAACAGCCATTATGGT-1-2dpf,cluster_cluster_1
AAACATGCATAATCGT-1-2dpf,2dpf,405.0,290,1.975309,313.0,162,2dpf,1434.0,350,0.511921,...,cluster_1,cluster_1,cluster_1,cluster_1,cluster_1,cluster_1,2df_AAACATGCATAATCGT-1,AAACATGCATAATCGT-1,AAACATGCATAATCGT-1-2dpf,cluster_cluster_5
AAACCAACAACCGCCA-1-2dpf,2dpf,538.0,434,2.230483,2621.0,1326,2dpf,1459.0,476,0.585348,...,cluster_12,cluster_12,cluster_12,cluster_12,cluster_12,cluster_2,2df_AAACCAACAACCGCCA-1,AAACCAACAACCGCCA-1,AAACCAACAACCGCCA-1-2dpf,cluster_cluster_4
AAACCAACAAGACTCC-1-2dpf,2dpf,1134.0,795,1.410935,4188.0,2120,2dpf,1594.0,801,0.586019,...,cluster_9,cluster_9,cluster_9,cluster_8,cluster_8,cluster_16,2df_AAACCAACAAGACTCC-1,AAACCAACAAGACTCC-1,AAACCAACAAGACTCC-1-2dpf,cluster_cluster_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGGCATAATCCG-1-6dpf,6dpf,1403.0,850,0.498931,6283.0,2960,6dpf,1667.0,856,0.548053,...,cluster_3,cluster_3,cluster_3,cluster_3,cluster_3,cluster_11,6df_TTTGTGGCATAATCCG-1,TTTGTGGCATAATCCG-1,TTTGTGGCATAATCCG-1-6dpf,cluster_cluster_7
TTTGTGGCATAGCTTG-1-6dpf,6dpf,868.0,552,1.612903,3720.0,1802,6dpf,1735.0,594,0.523116,...,cluster_5,cluster_5,cluster_5,cluster_4,cluster_4,cluster_12,6df_TTTGTGGCATAGCTTG-1,TTTGTGGCATAGCTTG-1,TTTGTGGCATAGCTTG-1-6dpf,cluster_cluster_8
TTTGTGTTCACCGGTA-1-6dpf,6dpf,3819.0,987,1.675831,4408.0,2112,6dpf,2383.0,957,0.526085,...,cluster_28,cluster_29,cluster_29,cluster_31,cluster_30,cluster_4,6df_TTTGTGTTCACCGGTA-1,TTTGTGTTCACCGGTA-1,TTTGTGTTCACCGGTA-1-6dpf,cluster_cluster_0


In [9]:
# Path to regions
#path_to_regions = outDir + 'consensus_peak_calling/consensus_regions.bed'

path_to_regions = {'2dpf':outDir + 'consensus_peak_calling/consensus_regions.bed',
                   '4dpf':outDir + 'consensus_peak_calling/consensus_regions.bed',
                   '6dpf':outDir + 'consensus_peak_calling/consensus_regions.bed'}

# Blacklist
#path_to_blacklist = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/pycisTopic/blacklist/hg38-blacklist.v2.bed'


In [10]:

# Metrics
infile = open(outDir + 'quality_control/metadata_bc.pkl', 'rb')
metadata_bc = pickle.load(infile)
infile.close()


In [11]:

# Valid barcodes
infile = open(outDir + 'quality_control/bc_passing_filters.pkl', 'rb')
bc_passing_filters = pickle.load(infile)
infile.close()


In [12]:
#Create objects
from pycisTopic.cistopic_class import *
cistopic_obj_list=[create_cistopic_object_from_fragments(path_to_fragments=fragments_dict[key],
                                               path_to_regions=path_to_regions[key],
                                               metrics=metadata_bc[key],
                                               valid_bc=bc_passing_filters[key],
                                               n_cpu=26,
                                               project=key,
                                               split_pattern='-') for key in fragments_dict.keys()]

2024-04-04 00:47:10,735 cisTopic     INFO     Reading data for 6dpf
2024-04-04 00:48:21,105 cisTopic     INFO     metrics provided!
2024-04-04 00:48:24,412 cisTopic     INFO     valid_bc provided, selecting barcodes!
2024-04-04 00:48:28,786 cisTopic     INFO     Counting fragments in regions
2024-04-04 00:48:32,279 root         INFO     Unable to poll TPU GCE metadata: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/attributes/accelerator-type (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f50f697fb20>: Failed to establish a new connection: [Errno -2] Name or service not known'))


2024-04-04 00:48:33,593	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


2024-04-04 00:50:13,407 cisTopic     INFO     Creating fragment matrix
2024-04-04 00:51:10,358 cisTopic     INFO     Converting fragment matrix to sparse matrix
2024-04-04 00:51:15,743 cisTopic     INFO     Creating CistopicObject
2024-04-04 00:51:16,639 cisTopic     INFO     Done!
2024-04-04 00:51:17,126 cisTopic     INFO     Reading data for 4dpf
2024-04-04 00:52:26,144 cisTopic     INFO     metrics provided!
2024-04-04 00:52:33,923 cisTopic     INFO     valid_bc provided, selecting barcodes!
2024-04-04 00:52:39,104 cisTopic     INFO     Counting fragments in regions
2024-04-04 00:52:42,956 root         INFO     Unable to poll TPU GCE metadata: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/attributes/accelerator-type (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f46e49ed250>: Failed to establish a new connection: [Errno -2] Name or service not known'))


2024-04-04 00:52:43,963	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


2024-04-04 00:54:18,286 cisTopic     INFO     Creating fragment matrix
2024-04-04 00:54:41,168 cisTopic     INFO     Converting fragment matrix to sparse matrix
2024-04-04 00:54:43,805 cisTopic     INFO     Creating CistopicObject
2024-04-04 00:54:44,552 cisTopic     INFO     Done!
2024-04-04 00:54:44,916 cisTopic     INFO     Reading data for 2dpf
2024-04-04 00:55:34,834 cisTopic     INFO     metrics provided!
2024-04-04 00:55:38,592 cisTopic     INFO     valid_bc provided, selecting barcodes!
2024-04-04 00:55:41,138 cisTopic     INFO     Counting fragments in regions
2024-04-04 00:55:45,803 root         INFO     Unable to poll TPU GCE metadata: HTTPConnectionPool(host='metadata.google.internal', port=80): Max retries exceeded with url: /computeMetadata/v1/instance/attributes/accelerator-type (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f494b101400>: Failed to establish a new connection: [Errno -2] Name or service not known'))


2024-04-04 00:55:47,225	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


2024-04-04 00:57:35,677 cisTopic     INFO     Creating fragment matrix
2024-04-04 00:58:11,772 cisTopic     INFO     Converting fragment matrix to sparse matrix
2024-04-04 00:58:18,432 cisTopic     INFO     Creating CistopicObject
2024-04-04 00:58:19,793 cisTopic     INFO     Done!


In [13]:
cistopic_obj=merge(cistopic_obj_list)

# Save
with open(outDir + 'cisTopicObject.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

2024-04-04 01:01:03,737 cisTopic     INFO     cisTopic object 1 merged
2024-04-04 01:01:09,669 cisTopic     INFO     cisTopic object 2 merged


In [14]:

infile = open(outDir + 'cisTopicObject.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()

In [15]:

cistopic_obj.add_cell_data(cell_df, split_pattern='___')

In [16]:

# Save with doublets
with open(outDir + 'cisTopicObject.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

In [17]:
# Save
with open(outDir + 'cisTopicObject_noDBL.pkl', 'wb') as f:
  pickle.dump(cistopic_obj, f)

: 